In [1]:
import os
import re
import pickle5 as pickle
import pandas as pd

In [140]:
with open('/mnt/transcriber/sales_test') as f:
    arr1 = f.readlines()
arr1 = [id.strip("\n") for id in arr1]

with open('/mnt/transcriber/test_ids.txt') as f:
    arr2 = f.readlines()
arr2 = [id.strip("\n") for id in arr2]

In [141]:
arr1 = set(arr1)

In [2]:
path = '/mnt/transcriber/Call_Scoring/transcriptions/single_ch_b1/single_ch/'
ids = set()
for file in os.listdir(path):
    id = re.split("_|-|\.", file)[1]
    ids.add(id)

In [3]:
len(ids)

5945

In [4]:
path = '/mnt/transcriber/Call_Scoring/transcriptions/customer_ch/test/'
cust_test = set()
for file in os.listdir(path):
    id = re.split("_|-|\.", file)[1]
    cust_test.add(id)

In [5]:
len(cust_test)

2993

In [6]:
path = '/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/train/'
csr_train = set()
for file in os.listdir(path):
    id = re.split("_|-|\.", file)[1]
    csr_train.add(id)

In [20]:
path = '/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/test/'
csr_test = set()
for file in os.listdir(path):
    id = re.split("_|-|\.", file)[1]
    csr_test.add(id)

In [21]:
len(csr_test)

2407

In [2]:
def prepare_score_df(path_to_p, workgroup):
    with open(path_to_p, 'rb') as file:
        df = pickle.load(file)
    df = df.sort_values(by=['RecordingDate', 'QGroupSequence', 'QuestionSequence']).copy()
    cols = ['QGroupSequence', 'QGroupName', 'InteractionIdKey', 'QuestionSequence', 'QuestionText', 'QuestionType',
            'QuestionPromptType', 'QuestionWeight', 'QuestionMin', 'QuestionMax', 'AnswerScore', 'RawAnswer',
            'DisplayAnswer',
            'UserComments']
    if workgroup == 'all':
        calls_df = df[(df.QuestionnaireName == 'Call Interaction') & ((df.WorkgroupQueue == 'Customer Service')
                                                                      |(df.WorkgroupQueue == 'Sales'))].copy()
    elif workgroup == 'CustomerService':
        calls_df = df[(df.QuestionnaireName == 'Call Interaction') & (df.WorkgroupQueue == 'Customer Service')].copy()
    elif workgroup == 'Sales':
        calls_df = df[(df.QuestionnaireName == 'Call Interaction') & (df.WorkgroupQueue == 'Sales')].copy()

    q_df = calls_df[cols]
    temp = q_df[0:10]
    temp = temp.reset_index(drop=True)
    q_text = []
    for index, row in temp.iterrows():
        q_text.append(row['QuestionText'])

    score_df = pd.DataFrame()
    score_df['WorkgroupQueue'] = calls_df.WorkgroupQueue[::10]
    score_df['RecordingDate'] = calls_df.RecordingDate[::10]
    # change baseline score range accordingly
    # overall score between 0 and 1
    score_df['CombinedPercentileScore'] = (calls_df.CombinedPercentileScore[::10]/100).astype(float).round(4)
    score_df['Category'] = (score_df['CombinedPercentileScore'] < 0.75).apply(lambda x: int(x))
    # overall score between 0 and 100
    # score_df['CombinedPercentileScore'] = (calls_df.CombinedPercentileScore[::10]).astype(float).round(2)
    # score_df['Category'] = (score_df['CombinedPercentileScore'] > 75).apply(lambda x: int(x))
    score_df.index = calls_df.InteractionIdKey[::10]
    calls_df.AnswerScore = calls_df.AnswerScore.astype('int')
    for i in range(10):
        criteria = q_text[i]
        q_max = int(calls_df.QuestionMax.iloc[i])
        score_df[criteria] = (calls_df.AnswerScore[i::10]).values
        score_df[criteria] = score_df[criteria].apply(lambda x: 0 if x >= q_max else 1) #used as binary class
        # score_df[criteria] = score_df[criteria].apply(lambda x: 1 if x >= q_max else 0) #used as binary class
        score_df[criteria + ' Feedback'] = (calls_df.UserComments[i::10]).values
    score_df = score_df.loc[~score_df.index.duplicated(keep='last')]
    print("Dataframe creation done")

    return score_df, q_text

In [3]:
path = '/mnt/transcriber/Call_Scoring/ScoringDetail_viw_all_subscore.p'
score_df, qt = prepare_score_df(path, 'all')

Dataframe creation done


In [11]:
with open(path, 'rb') as file:
    df = pickle.load(file)
df.columns

Index(['RecordingId', 'RecordingDate', 'MediaType', 'InteractionIdKey',
       'RecordingLength', 'InitiationPolicyName', 'Direction',
       'RecordedUserIndivId', 'RecordedUserLastName', 'RecordedUserFirstName',
       'RecordedUserMiddleName', 'RecordedUserDisplayName',
       'RecordedUserICUserId', 'WorkgroupQueue', 'QuestionnaireId',
       'QuestionnaireName', 'QuestionnaireNote', 'QuestionnaireMinScore',
       'QuestionnaireMaxScore', 'QDirectoryName', 'QGroupId', 'QGroupSequence',
       'QGroupName', 'QGroupNote', 'IsOptional', 'QuestionId',
       'QuestionSequence', 'QuestionText', 'QuestionType',
       'QuestionPromptType', 'QuestionWeight', 'QuestionMin', 'QuestionMax',
       'QuestionMinAcceptableScore', 'QuestionCanMarkNA', 'FormId',
       'ScoredUserLastName', 'ScoredUserFirstName', 'ScoredUserMiddleName',
       'ScoredUserICUserId', 'ScoredUserIndivID', 'ScoringUserLastName',
       'ScoringUserFirstName', 'ScoringUserMiddleName', 'ScoringUserICUserId',
       'S

In [4]:
score_df.WorkgroupQueue.value_counts()

Sales               12942
Customer Service     9437
Name: WorkgroupQueue, dtype: int64

In [7]:
score_df.RecordingDate.max()

Timestamp('2021-08-26 13:54:01.867000')

In [4]:
import os
import re
path = '/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/train/'
ids = set()
for file in os.listdir(path):
    id = re.split("_|-|\.", file)[1]
    ids.add(id)

In [22]:
csr_test = ([id for id in csr_test if id in score_df.index])
csr_df = score_df.loc[csr_test,:]

In [23]:
len(csr_df)

2404

In [19]:
csr_df.WorkgroupQueue.value_counts()

Sales               2404
Customer Service     189
Name: WorkgroupQueue, dtype: int64

In [16]:
mv = csr_df[csr_df.WorkgroupQueue=="Customer Service"].index

In [17]:
len(mv)

189

In [13]:
rem_cs_ids = set(cs_df.index).difference(ids)

In [14]:
len(rem_cs_ids)

7160

In [15]:
for id in rem_cs_ids:
    file = "/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/train/IRCall_"+id+'_singlech.txt'
    if os.path.exists(file):
        print("Found file")

In [18]:
for id in mv:
    file = "/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/test/IRCall_"+id+'_singlech.txt'
    if os.path.exists(file):
        os.rename(file,"/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/train/IRCall_"+id+'_singlech.txt' )

In [7]:
ids = ([id for id in ids if id in score_df.index])
df = score_df.loc[ids,:]

In [8]:
cs_ids = df[df.WorkgroupQueue=='Customer Service'].index

In [9]:
len(cs_ids)

2277

In [20]:
for id in cs_ids:
    source = "/mnt/transcriber/Call_Scoring/transcriptions/single_ch_b1/single_ch/"+'IRCall_'+id+'_singlech.txt'
    dest = "/mnt/transcriber/Call_Scoring/transcriptions/csr_ch/train/"+'IRCall_'+id+'_singlech.txt'
    os.rename(source, dest)